In [30]:
import pandas as pd
import numpy as np
from keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import scipy.stats as stats
import time
import csv
# import busio
# import adafruit_adxl34x
import math


In [31]:
fs=20
frame_size=fs*10
hop_size=fs*2

def get_frames(df,frame_size,hop_size):
    N_FEATURES=3
    frames=[]
    lables=[]
    for i in range(0,len(df)-frame_size,hop_size):
        x=df['x-axis'].values[i: i+frame_size]
        y=df['y-axis'].values[i: i+frame_size]
        z=df['z-axis'].values[i: i+frame_size]   
        
        lable = stats.mode(df['number'][i: i+frame_size])[0][0]
        frames.append([x,y,z])
        lables.append(lable)
    frames=np.asarray(frames).reshape(-1,frame_size,N_FEATURES)
    lables=np.asarray(lables)
    
    return frames ,lables

In [37]:
f=pd.read_csv('3.csv')
df=pd.DataFrame(f)
columns=['x-axis','y-axis','z-axis','number']
df.columns=columns
df

,x-axis,y-axis,z-axis,number
0,5.839575,3.462974,-4.725073,3
1,6.909759,1.990709,-2.160835,3
2,8.178706,3.887233,-1.015505,3
3,8.032903,-0.513226,-1.267771,3
4,14.695878,-1.371356,-4.767555,3
...,...,...,...,...
89994,0.198186,-19.142152,15.008114,3
89995,-10.358698,1.879607,14.794481,3
89996,2.820141,-7.553847,-7.744405,3
89997,-15.877624,-7.054193,-0.982989,3


In [41]:
X_x, Y_y = get_frames(df, frame_size, hop_size)
s = list(X_x.shape)
X_X = X_x.reshape(s[0], s[1], s[2], 1)
X_X[:-1].shape

/var/folders/tn/c2cbg8n95ws2j0pgvtymb0cc0000gn/T/ipykernel_30943/1563789309.py:14: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  lable = stats.mode(df['number'][i: i+frame_size])[0][0]


(2244, 200, 3, 1)

In [38]:
model = load_model('model.h5', compile=False)
model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [40]:
p = model.predict(X_X[:-1])
a = max((p.max(axis=0)))
print("MAX", a)
print("Predicted values\n", p)
s = np.where(p == a)  # index of the max
l = list(s)
print("Predected number is: ", int(l[1]))

20/20 [==============================] - 0s 16ms/step
MAX 0.99953973
Predicted values
 [[2.0296570e-02 3.0474639e-01 1.1792052e-06 ... 2.9272097e-01
  1.1506746e-06 3.3293200e-01]
 [6.8623968e-03 5.8967549e-01 2.5609027e-06 ... 2.8419864e-01
  3.1204874e-07 2.3964649e-02]
 [8.3849188e-03 6.8867594e-02 4.6704314e-05 ... 6.3067472e-01
  1.8090819e-07 1.2209103e-03]
 ...
 [6.7579746e-01 1.2013047e-14 2.2986564e-03 ... 2.1206986e-03
  2.3384038e-03 1.7237720e-07]
 [4.3809670e-01 1.2788994e-15 5.6642992e-04 ... 3.3383994e-04
  5.6594238e-03 2.5819299e-08]
 [5.6628245e-01 3.7332767e-14 7.5275393e-04 ... 9.0816425e-04
  1.5094316e-02 1.5896045e-07]]
Predected number is:  0
